In [37]:
import pandas as pd
import numpy as np
import math
from pulp import *



#Random
np.random.seed(20061061)



#Factory to Distribution Center(per unit)

RandomFtoD1=np.round(np.random.uniform(low=1,high=10,size=12),0).astype(int)
RandomFtoD2=np.reshape(RandomFtoD1,(4,3))
#(Q2/e)-RandomFtoD2[0][0]=10
CostFtoD=pd.DataFrame(RandomFtoD2,columns=['Atlas','Nebule','Azure'],
                      index=['Tonoz','White Kiosk','Quarterage','Middle Yard'])



#Distribution Center to Shop(per unit)

RandomDtoS1=np.round(np.random.uniform(low:=1, high=10, size=15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1,(3,5))
#(Q2/e)-RandomDtoS2[2][1]=5
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
index=['Atlas','Nebule','Azure']) 



#Factory to Shop(per unit)

RandomFtoS1=np.round(np.random.uniform(low = 1,  high = 10,size=20),0).astype(int)
RandomFtoS2 =np.reshape(RandomFtoS1,(4,5))
CostFtoS=pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
index=['Tonoz','White Kiosk','Quarterage','Middle Yard'])



#Supply Information

supply=[3000,  #Tonoz
        5000,  #White Kiosk
        10000, #Quarterage (Q2/d)-7500
        7000]  #Middle Yard (Q2/f)-5000


#Demand Information

demand=[5500, #Venus
        4750, #Minerva
        6550, #Neptunus
        4000, #Mars
        4200] #Ceres (Q2/d)-1700



print("Factory to Distribution Center Cost (per unit) ")
print("-------------------")
print(CostFtoD)
print("Distribution Center to Shop Cost(per unit)")
print("-------------------")
print(CostDtoS)
print("Factory to Shop Cost(per unit)")
print("-------------------")
print(CostFtoS)



Factory to Distribution Center Cost (per unit) 
-------------------
             Atlas  Nebule  Azure
Tonoz            5       1      5
White Kiosk      6       2      8
Quarterage       4       3      9
Middle Yard      8       5      2
Distribution Center to Shop Cost(per unit)
-------------------
        Venus  Minerva  Neptunus  Mars  Ceres
Atlas       8        2         3     8      4
Nebule      5        4         7     6      2
Azure       2        3         3     1      9
Factory to Shop Cost(per unit)
-------------------
             Venus  Minerva  Neptunus  Mars  Ceres
Tonoz            9        2         7     6      8
White Kiosk      2        9         8     4      6
Quarterage       3       10         9     2      4
Middle Yard      2        6         3     5      8


In [38]:
import pandas as pd
import numpy as np
import math
from pulp import *



#Objective Function

prob = LpProblem('Supply_Chain_Optimization', LpMinimize)



#Decision Variables

#Factory To Distribution Center

FD = [LpVariable("x{0}{1}".format(i+1, j+1),0, cat="Continuous")
          for i in range(4) for j in range(3)]



#Distribution Center to Shop

DS = [LpVariable("y{0}{1}".format(j+1, k+1),0, cat="Continuous")
          for j in range(3) for k in range(5)]



#Factory to Shop

FS = [LpVariable("z{0}{1}".format(i+1, k+1),0, cat="Continuous")
          for i in range(4) for k in range(5)]




#Objective Function

prob += lpSum(CostFtoD.iloc[i, j] * FD[i * 3 + j]
              for i in range(4)
              for j in range(3)) + lpSum(CostDtoS.iloc[j, k] * DS[k * 3 + j]
                                        for j in range(3)
                                        for k in range(5)) + lpSum(CostFtoS.iloc[i, k] * FS[i * 5 + k]
                                                                    for i in range(4)
                                                                    for k in range(5))


           
#Supply Constraints

for i in range(4):
    sup_X=0
    sup_Z=0
    for j in range(3):
        sup_X+=FD[i*3+j]
    for k in range(5):
        sup_Z+=FS[i*5+k]
    prob+=sup_X+sup_Z<=supply[i]
    
    

#Demand Constraints

for k in range(5):
    dem_Y=0
    dem_Z=0
    for j in range(3):
        dem_Y+=DS[5*j+k]
    for i in range(4):
        dem_Z+=FS[5*i+k]
    prob+=dem_Y+dem_Z>=demand[k]



#Products which had come to Distribution Center must be more than the product number which is going to be shipped from Dist. C.  

for j in range(3):
    dst_from_X=0
    dst_to_Y=0
    for i in range(4):
        dst_from_X+=FD[3*i+j]
    for k in range(5):
        dst_to_Y+=DS[j*5+k]
    prob+=dst_from_X>=dst_to_Y



#Problem && Solving

print(prob)
prob.solve()



#Objective Function's Value

print(value(prob.objective),"/",LpStatus[prob.solve()])



#Variable's values

for v in prob.variables():
    print(v.name, "=", v.varValue)

    
    

Supply_Chain_Optimization:
MINIMIZE
5*x11 + 1*x12 + 5*x13 + 6*x21 + 2*x22 + 8*x23 + 4*x31 + 3*x32 + 9*x33 + 8*x41 + 5*x42 + 2*x43 + 8*y11 + 5*y12 + 2*y13 + 2*y14 + 4*y15 + 3*y21 + 3*y22 + 7*y23 + 3*y24 + 8*y25 + 6*y31 + 1*y32 + 4*y33 + 2*y34 + 9*y35 + 9*z11 + 2*z12 + 7*z13 + 6*z14 + 8*z15 + 2*z21 + 9*z22 + 8*z23 + 4*z24 + 6*z25 + 3*z31 + 10*z32 + 9*z33 + 2*z34 + 4*z35 + 2*z41 + 6*z42 + 3*z43 + 5*z44 + 8*z45 + 0
SUBJECT TO
_C1: x11 + x12 + x13 + z11 + z12 + z13 + z14 + z15 <= 3000

_C2: x21 + x22 + x23 + z21 + z22 + z23 + z24 + z25 <= 5000

_C3: x31 + x32 + x33 + z31 + z32 + z33 + z34 + z35 <= 10000

_C4: x41 + x42 + x43 + z41 + z42 + z43 + z44 + z45 <= 7000

_C5: y11 + y21 + y31 + z11 + z21 + z31 + z41 >= 5500

_C6: y12 + y22 + y32 + z12 + z22 + z32 + z42 >= 4750

_C7: y13 + y23 + y33 + z13 + z23 + z33 + z43 >= 6550

_C8: y14 + y24 + y34 + z14 + z24 + z34 + z44 >= 4000

_C9: y15 + y25 + y35 + z15 + z25 + z35 + z45 >= 4200

_C10: x11 + x21 + x31 + x41 - y11 - y12 - y13 - y14 - y15 >= 0
